In [3]:
# fix_chart_paths.py — run from anywhere
from pathlib import Path
from datetime import datetime
import json, re

INSIGHTS = Path("/Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/app/data/insights.json")

data = json.loads(INSIGHTS.read_text(encoding="utf-8"))

def to_web_path(chart_path: str) -> str:
    if not chart_path:
        return chart_path
    # If it already starts with /charts/, keep it
    if chart_path.startswith("/charts/"):
        return chart_path
    # If it's an absolute or repo-ish path containing /public/charts/..., convert to /charts/...
    m = re.search(r"/public/(charts/.*)$", chart_path)
    if m:
        return "/" + m.group(1)
    # Also handle cases like "genesis-research/public/charts/..."
    m = re.search(r"public/(charts/.*)$", chart_path)
    if m:
        return "/" + m.group(1)
    # Fallback: if it ends with charts/... assume that is the tail
    m = re.search(r"(charts/.*)$", chart_path)
    if m:
        return "/" + m.group(1)
    # otherwise leave as-is
    return chart_path

for ins in data:
    cp = ins.get("chartPath","")
    fixed = to_web_path(cp)
    if fixed != cp:
        ins["chartPath"] = fixed

# sort by date ASC and re-id
def key(ins):
    try:
        return datetime.strptime(ins.get("date","01-01-1900"), "%d-%m-%Y")
    except Exception:
        return datetime(1900,1,1)

data.sort(key=key)
for i, ins in enumerate(data, 1):
    ins["id"] = i

# backup + write
backup = INSIGHTS.with_suffix(".json.bak-chartpaths")
backup.write_text(INSIGHTS.read_text(encoding="utf-8"), encoding="utf-8")
INSIGHTS.write_text(json.dumps(data, indent=2, ensure_ascii=False), encoding="utf-8")

print("✅ Fixed chartPath fields to web paths.")
print(f"🛟 Backup at: {backup}")
print(f"💾 Updated:   {INSIGHTS}")


✅ Fixed chartPath fields to web paths.
🛟 Backup at: /Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/app/data/insights.json.bak-chartpaths
💾 Updated:   /Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/app/data/insights.json
